In [ ]:
import numpy as np
import training_data_generation as tdg
import matplotlib.pyplot as plt
%matplotlib inline
plt.ion();
from matplotlib import interactive
interactive(True)

# Diagnostic

In [ ]:
import healpy as hp
import utils_intensity_map as uim
import utils_deck_generation as idg
import netcdf_read_write as nrw
import training_data_generation as tdg
import utils_healpy as uhp
import os
%matplotlib inline
np_complex = np.vectorize(complex)


In [ ]:
diag_dir = "../Data/data_file"
iex = 0
ind_profile = 0

with_pointing_markers = True
import_flipped = False
old_format = False
display_steradians = False
sys_params = tdg.define_system_params(diag_dir)

print_list1 = []
print_list2 = []
print_list3 = []

## Analyse optimization outputs

In [ ]:
dataset_params = nrw.read_general_netcdf(sys_params["data_dir"] + "/" + sys_params["dataset_params_filename"])
facility_spec = nrw.read_general_netcdf(sys_params["data_dir"] + "/" + sys_params["facility_spec_filename"])
dataset = nrw.read_general_netcdf(sys_params["data_dir"]+"/"+sys_params["trainingdata_filename"])

avg_flux = dataset["avg_flux"][iex, ind_profile]
real_modes = dataset["real_modes"][iex,ind_profile,:]
imag_modes = dataset["imag_modes"][iex,ind_profile,:]
rms = dataset["rms"][iex, ind_profile]

intensity_map_normalized = uhp.modes2imap(real_modes, imag_modes, dataset_params["imap_nside"])
intensity_map_sr = (intensity_map_normalized+1)*avg_flux

if dataset_params["run_plasma_profile"]:
    drive_map = intensity_map_normalized * 100.
    drive_units = r"Ablation pressure {:.0f} Mbar ($ \pm \%$)".format(avg_flux)
else:
    drive_map = intensity_map_normalized * 100.
    if display_steradians:
        drive_units = r"Intensity {:.1e} $ \rm W/sr   (\pm \%) $".format(avg_flux)
    else:
        drive_units = r"Intensity {:.1e} $ \rm W/cm^2 (\pm \%) $".format(avg_flux / (dataset_params['illumination_evaluation_radii'][ind_profile] / 10000.0)**2)

mapmin = np.round(np.min(drive_map), +1-int(np.floor(np.log10(abs(np.min(drive_map))))))
mapmax = np.round(np.max(drive_map), +1-int(np.floor(np.log10(abs(np.max(drive_map))))))
hp.mollview(drive_map, unit=drive_units,flip="geo", min=mapmin, max=mapmax)
hp.graticule()
plt.savefig(diag_dir+'/intensity_mollweide_'+str(iex) + "_" + str(ind_profile) + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

complex_modes = np_complex(real_modes, imag_modes)
power_spectrum = uhp.alms2power_spectrum(complex_modes, dataset_params["LMAX"])

LMAX = dataset_params["LMAX"]
fig = plt.figure()
ax = plt.axes()
plt.plot(np.arange(LMAX), np.sqrt(power_spectrum) * 100.0)
ax.set_xticks(range(0, LMAX+1, int(LMAX/5)))
plt.xlim([0, LMAX])
plt.title("Intensity Modes")
plt.xlabel("l mode")
plt.ylabel(r"amplitude ($\%$)")
plt.savefig(diag_dir+"/modes_"+str(iex) + "_" + str(ind_profile) + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

In [ ]:
if with_pointing_markers:
    
    mapmin = np.round(np.min(drive_map), +1-int(np.floor(np.log10(abs(np.min(drive_map))))))
    mapmax = np.round(np.max(drive_map), +1-int(np.floor(np.log10(abs(np.max(drive_map))))))
    hp.mollview(drive_map, unit=drive_units,flip="geo", min=mapmin, max=mapmax)
    hp.graticule()
    
    deck_gen_params = nrw.read_general_netcdf(sys_params["data_dir"] + "/" + sys_params["deck_gen_params_filename"])

    print_list1, power_deposited = uim.readout_intensity(facility_spec, intensity_map_sr, dataset_params, ind_profile)
    print_list2 = uim.extract_run_parameters(iex, ind_profile, power_deposited, dataset_params, facility_spec, sys_params, deck_gen_params)
    print_list = print_list1 + print_list2
    stats_filename = diag_dir+"/stats"+str(iex) + "_" + str(ind_profile)+".txt"
    uim.print_save_readout(print_list, stats_filename)

    port_theta = facility_spec["Theta"]
    port_phi = facility_spec["Phi"]

    pointing_theta = np.squeeze(deck_gen_params["theta_pointings"][iex,:])
    pointing_phi = np.squeeze(deck_gen_params["phi_pointings"][iex,:])

    num_ifriit_beams = int(facility_spec['nbeams'] / facility_spec['beams_per_ifriit_beam'])
    port_cartesian = np.zeros((num_ifriit_beams, 2))
    portc_cartesian = np.zeros((num_ifriit_beams, 2))
    pointing_cartesian = np.zeros((num_ifriit_beams, 2))
    
    for ibeam in range(0, num_ifriit_beams):
        port_cartesian[ibeam,0], port_cartesian[ibeam,1] = uim.angle2moll(facility_spec["Theta"][ibeam], facility_spec["Phi"][ibeam])
        portc_cartesian[ibeam,0], portc_cartesian[ibeam,1] = uim.angle2moll(deck_gen_params["port_centre_theta"][ibeam], deck_gen_params["port_centre_phi"][ibeam])
        
        if dataset_params["pointing_bool"]:
            pointing_cartesian[ibeam,0], pointing_cartesian[ibeam,1] = uim.angle2moll(pointing_theta[ibeam], pointing_phi[ibeam])
            dx = pointing_cartesian[ibeam,0] - port_cartesian[ibeam,0]
            dy = pointing_cartesian[ibeam,1] - port_cartesian[ibeam,1]
            # reduce line length to give space for arrow head
            head_length = 0.05
            shaft_angle = np.arctan2(dy, dx)
            shaft_length = np.sqrt(dx**2 + dy**2) - head_length
            if shaft_length > 0.0:
                dx = shaft_length * np.cos(shaft_angle)
                dy = shaft_length * np.sin(shaft_angle)

                if np.abs(dx) < 1.0: # don't plot arrow for pointings which cross the edge
                    plt.arrow(port_cartesian[ibeam,0], port_cartesian[ibeam,1], dx, dy, linewidth=3, head_width=head_length, head_length=head_length, fc='k', ec='k')
            else:
                plt.arrow(port_cartesian[ibeam,0], port_cartesian[ibeam,1], dx, dy)
            plt.scatter(pointing_cartesian[ibeam,0], pointing_cartesian[ibeam,1],c="white", marker="x")
    plt.scatter(port_cartesian[:,0], port_cartesian[:,1],c="white", marker="s")

In [ ]:
if (dataset_params["facility"] == "nif") or (dataset_params["facility"] == "lmj"):
    mapmin = np.round(np.min(drive_map), +1-int(np.floor(np.log10(abs(np.min(drive_map))))))
    mapmax = np.round(np.max(drive_map), +1-int(np.floor(np.log10(abs(np.max(drive_map))))))
    hp.mollview(drive_map, unit=drive_units,flip="geo", min=mapmin, max=mapmax)
    hp.graticule()

    deck_gen_params = nrw.read_general_netcdf(sys_params["data_dir"] + "/" + sys_params["deck_gen_params_filename"])

    quad_list = list(set(facility_spec["Quad"]))
    cone_list = list(set(facility_spec["Cone"]))

    port_theta = deck_gen_params["port_centre_theta"]
    port_phi = deck_gen_params["port_centre_phi"]

    pointing_theta = np.squeeze(deck_gen_params["theta_pointings"][iex,:])
    pointing_phi = np.squeeze(deck_gen_params["phi_pointings"][iex,:])

    num_ifriit_beams = int(facility_spec['nbeams'] / facility_spec['beams_per_ifriit_beam'])
    port_cartesian = np.zeros((num_ifriit_beams, 2))
    port_centre_cartesian = np.zeros((num_ifriit_beams, 2))
    pointing_cartesian = np.zeros((num_ifriit_beams, 2))
    pointing_centre_cartesian = np.zeros((num_ifriit_beams, 2))

    quad_name_plot_list = dataset_params['quad_from_each_group']

    quad_count = 0
    for quad_name in quad_list:
        quad_slice = np.where(quad_name == facility_spec["Quad"])[0]

        for ibeam in quad_slice:
            port_cartesian[ibeam,0], port_cartesian[ibeam,1] = uim.angle2moll(facility_spec["Theta"][ibeam], facility_spec["Phi"][ibeam])
            port_centre_cartesian[ibeam,0], port_centre_cartesian[ibeam,1] = uim.angle2moll(deck_gen_params["port_centre_theta"][ibeam], deck_gen_params["port_centre_phi"][ibeam])
            pointing_cartesian[ibeam,0], pointing_cartesian[ibeam,1] = uim.angle2moll(pointing_theta[ibeam], pointing_phi[ibeam])

        if quad_name in quad_name_plot_list:
            line_colour = ["r", "m", "c", "b"]
            colour_index = np.where(facility_spec['Cone'][quad_slice[0]] == cone_list)[0][0]
            for iquad in range(len(quad_slice)):
                dx = pointing_cartesian[quad_slice[iquad],0] - pointing_cartesian[quad_slice[iquad-1],0]
                if np.abs(dx) < 1.0:
                    plt.plot([pointing_cartesian[quad_slice[iquad],0], pointing_cartesian[quad_slice[iquad-1],0]], 
                             [pointing_cartesian[quad_slice[iquad],1], pointing_cartesian[quad_slice[iquad-1],1]], zorder=0, color=line_colour[colour_index])
        quad_count += 1

        theta_average = np.mean(pointing_theta[quad_slice])
        phi_average = np.mean(pointing_phi[quad_slice])
        pointing_centre_cartesian[quad_slice,0], pointing_centre_cartesian[quad_slice,1] = uim.angle2moll(theta_average, phi_average)

        dx = pointing_centre_cartesian[ibeam,0] - port_centre_cartesian[ibeam,0]
        dy = pointing_centre_cartesian[ibeam,1] - port_centre_cartesian[ibeam,1]
        # reduce line length to give space for arrow head
        head_length = 0.05
        shaft_angle = np.arctan2(dy, dx)
        shaft_length = np.sqrt(dx**2 + dy**2) - head_length
        if shaft_length > 0.0:
            dx = shaft_length * np.cos(shaft_angle)
            dy = shaft_length * np.sin(shaft_angle)

            if (np.abs(dx) < 1.0) and (np.std(pointing_phi[quad_slice]) < np.radians(40)): # don't plot arrow for pointings which cross the edge
                plt.arrow(portc_cartesian[ibeam,0], portc_cartesian[ibeam,1], dx, dy, linewidth=3, head_width=head_length, head_length=head_length, fc='k', ec='k', zorder=10)

    plt.scatter(port_cartesian[:,0], port_cartesian[:,1],c="white", marker="s", zorder=5)
    plt.scatter(pointing_cartesian[:,0], pointing_cartesian[:,1],c="white", marker="x", zorder=5)
    plt.savefig(diag_dir+'/intensity_mollweide_'+str(iex) + "_show_pointings_" + str(ind_profile) + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

## Analysis from ifriit output (no plasma)

In [ ]:
sys_params = tdg.define_system_params(diag_dir)
sys_params["run_gen_deck"] = False

if old_format:
    num_examples = 1
    dataset_params, facility_spec = tdg.define_dataset_params(int(num_examples), sys_params)
    facility_spec['target_radius'] = [2307.0]
else:
    dataset_params = nrw.read_general_netcdf(diag_dir + "/" + sys_params["dataset_params_filename"])
    facility_spec = nrw.read_general_netcdf(diag_dir + "/" + sys_params["facility_spec_filename"])
dataset_params["num_examples"] = 1
dataset_params["run_plasma_profile"] = False

config_location = sys_params["data_dir"] + "/" + sys_params["config_dir"] + str(iex)
run_location = config_location + "/" + sys_params["sim_dir"] + str(ind_profile)
print(run_location)

print(dataset_params['target_radius'], dataset_params['illumination_evaluation_radii'][ind_profile])

In [ ]:
parameters = nrw.read_general_netcdf(run_location + "/" + sys_params["ifriit_ouput_name"])
intensity_map = parameters["intensity"]
# convert from W/cm**2 to W/sr
intensity_map_sr = parameters["intensity"] * (dataset_params['illumination_evaluation_radii'][ind_profile] / 10000.0)**2

intensity_map_normalized, avg_flux = uim.imap_norm(intensity_map)
rms = uim.extract_rms(intensity_map_normalized)

print_list1, power_deposited = uim.readout_intensity(facility_spec, intensity_map_sr, dataset_params, ind_profile)
stats_filename = run_location+"/stats"+str(iex)+".txt"
print_list = print_list1 + print_list2 + print_list3
uim.print_save_readout(print_list1, stats_filename)

if display_steradians:
    drive_map = intensity_map_sr
    drive_units=r"Intensity {:.1e} $\rm{W/sr}$".format(np.mean(intensity_map_sr))
else:
    drive_map = intensity_map_normalized*100.
    drive_units=r"Intensity {:.1e} $\rm W/cm^2 $ ($ \pm \%$)".format(avg_flux)
    
mapmin = np.round(np.min(drive_map), +1-int(np.floor(np.log10(abs(np.min(drive_map))))))
mapmax = np.round(np.max(drive_map), +1-int(np.floor(np.log10(abs(np.max(drive_map))))))
hp.mollview(drive_map, unit=drive_units, flip="geo", min=mapmin, max=mapmax)
hp.graticule()
plt.savefig(run_location+"/init_illumination" + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

In [ ]:
LMAX = dataset_params["LMAX"]
power_spectrum_unweighted = uhp.power_spectrum(intensity_map, LMAX)
x_max = LMAX

fig = plt.figure()
ax = plt.axes()
plt.plot(np.arange(LMAX), power_spectrum_unweighted * 100.0)
ax.set_xticks(range(0, LMAX+1, int(LMAX/5)))
plt.xlim([0, x_max])
plt.title("Intensity Modes")
plt.xlabel("l mode")
plt.ylabel(r"amplitude ($\%$)")
plt.savefig(run_location+"/init_modes" + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

In [ ]:
## Sanity check
intensity_map_normalized, avg_power = uim.imap_norm(intensity_map)
#hp.mollview((intensity_map_normalized+1)*avg_power, unit=r"$\rm{W/cm^2}$", flip="geo")

real_modes, imag_modes = uhp.imap2modes(intensity_map_normalized, LMAX)
intensity_map_normalized2 = uhp.modes2imap(real_modes, imag_modes, dataset_params["imap_nside"])
#hp.mollview((intensity_map_normalized2+1)*avg_power, unit=r"$\rm{W/cm^2}$", flip="geo")

## Estimate for initial ablation pressure at 351nm

In [ ]:
intensity_map_normalized, avg_power = uim.imap_norm(intensity_map)
intensity_map_init = (intensity_map_normalized+1) * avg_power
press_abla = 24.7 * (intensity_map_init / 1.0e14)**(2.0/3.0)
#hp.mollview(press_abla, unit=r"$\rm{Mbars}$", flip="geo")
press_abla_mean = np.mean(press_abla)
init_press_abla_rms = np.sqrt(np.mean((press_abla / press_abla_mean - 1.0)**2)) *100.0
print('Initial ablation pressure RMS: {:.2f}%, '.format(init_press_abla_rms))
print('Initial mean ablation pressure: {:.2f}Mbar, '.format(press_abla_mean))

## Open Ifriit heat source modes (with plasma)

In [ ]:
sys_params = tdg.define_system_params(diag_dir)
sys_params["run_gen_deck"] = False

if old_format:
    num_examples = 1
    target_radius_microns = [2307.0]
    dataset_params, facility_spec = tdg.define_dataset_params(int(num_examples), sys_params)
    facility_spec['target_radius'] = target_radius_microns
else:
    dataset_params = nrw.read_general_netcdf(diag_dir + "/" + sys_params["dataset_params_filename"])
    facility_spec = nrw.read_general_netcdf(diag_dir + "/" + sys_params["facility_spec_filename"])
dataset_params["num_examples"] = 1
dataset_params["run_plasma_profile"] = True

config_location = sys_params["data_dir"] + "/" + sys_params["config_dir"] + str(iex)
run_location = config_location + "/" + sys_params["sim_dir"] + str(ind_profile)

In [ ]:
file_name = sys_params["heat_source_nc"]
hs_and_modes = nrw.read_general_netcdf(run_location+"/"+file_name)

In [ ]:
avg_flux = hs_and_modes["average_flux"][0]
#print('Mean ablation pressure: {:.2f}Mbar, '.format(avg_flux))

lmax = 30
np_complex = np.vectorize(complex)
X_train_complex = np_complex(hs_and_modes["complex_modes"][0,:], hs_and_modes["complex_modes"][1,:])

intensity_map_normalized = hp.alm2map(X_train_complex, lmax)

drive_map = intensity_map_normalized * 100.
drive_units = r"Ablation pressure {:.0f} Mbar ($ \pm \%$)".format(avg_flux)
mapmin = np.round(np.min(drive_map), +1-int(np.floor(np.log10(abs(np.min(drive_map))))))
mapmax = np.round(np.max(drive_map), +1-int(np.floor(np.log10(abs(np.max(drive_map))))))
hp.mollview(drive_map, unit=drive_units,flip="geo", min=mapmin, max=mapmax)
hp.graticule()
plt.savefig(run_location+"/ablation_pressure" + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

print_list2, power_deposited = uim.readout_intensity(facility_spec, (intensity_map_normalized+1)*avg_flux, dataset_params, ind_profile)
print_list = print_list1 + print_list2 + print_list3
stats_filename = run_location+"/stats"+str(iex)+".txt"
uim.print_save_readout(print_list, stats_filename)

power_spectrum = uhp.alms2power_spectrum(X_train_complex, lmax)
#print("The rms direct from output modes: ", np.sqrt(np.sum(power_spectrum))*100.0, "%")

fig = plt.figure()
ax = plt.axes()
plt.plot(np.arange(lmax), np.sqrt(power_spectrum) * 100.0)
ax.set_xticks(range(0, lmax+1, int(lmax/5)))
plt.xlim([0, lmax])
plt.title("Pressure Modes")
plt.xlabel("l mode")
plt.ylabel(r"amplitude ($\%$)")
plt.savefig(run_location+"/ablation_pressure_modes" + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

In [ ]:
theta_edges = (hs_and_modes["theta"][1:] + hs_and_modes["theta"][:-1]) / 2.0
theta_edges = np.append(0.0, theta_edges)
theta_edges = np.append(theta_edges, np.pi)
phi_edges = (hs_and_modes["phi"][1:] + hs_and_modes["phi"][:-1]) / 2.0
phi_edges = np.append(0.0, phi_edges)
phi_edges = np.append(phi_edges, 2.0*np.pi)

theta_grid, phi_grid = np.meshgrid(theta_edges, phi_edges)

dphi = phi_grid[1:,1:] - phi_grid[:-1,1:]
d_cos_theta = np.cos(theta_grid[1:,1:]) - np.cos(theta_grid[1:,:-1])
domega = np.abs(dphi * d_cos_theta)

total_pwr = np.sum(hs_and_modes["heat_source"] * domega)
surface_area = 4.0 * np.pi * (dataset_params['illumination_evaluation_radii'][ind_profile] / 10000.0)**2
avg_flux = total_pwr / (4.0 * np.pi)

print('Mean intensity is {:.2e}W/cm^2, '.format(total_pwr / surface_area)) #/ 4.0 / np.pi
print('Intensity per steradian, {:.2e}W/sr'.format(avg_flux))

In [ ]:
theta_grid, phi_grid = np.meshgrid(hs_and_modes["theta"], hs_and_modes["phi"])
plt.pcolor(theta_grid, phi_grid, hs_and_modes["heat_source"])
plt.colorbar()

nside_low = 16
nside = 32

pixel_ind = hp.ang2pix(nside_low, theta_grid.reshape(-1), phi_grid.reshape(-1))
imap_low = np.zeros(hp.nside2npix(nside_low))
imap_low[pixel_ind] = hs_and_modes["heat_source"].reshape(-1)
imap_upgrade = hp.ud_grade(imap_low, nside)

pix_ind = hp.ang2pix(nside, theta_grid.reshape(-1), phi_grid.reshape(-1))
imap_high = np.zeros(hp.nside2npix(nside))
imap_upgrade[pix_ind] = hs_and_modes["heat_source"].reshape(-1)
avg_flux = np.mean(imap_upgrade) / (dataset_params['illumination_evaluation_radii'][ind_profile] / 10000.0)**2

drive_map = intensity_map_normalized * 100.
drive_units = r"Intensity {:.1e} $\rm W/cm^2 $ ($ \pm \%$)".format(avg_flux)
mapmin = np.round(np.min(drive_map), +1-int(np.floor(np.log10(abs(np.min(drive_map))))))
mapmax = np.round(np.max(drive_map), +1-int(np.floor(np.log10(abs(np.max(drive_map))))))
hp.mollview(drive_map, unit=drive_units,flip="geo", min=mapmin, max=mapmax)
hp.graticule()
plt.savefig(run_location+"/heatsource" + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

dataset_params["run_plasma_profile"] = False
print_list3, power_deposited = uim.readout_intensity(facility_spec, imap_upgrade, dataset_params, ind_profile)
print_list = print_list1 + print_list2 + print_list3
stats_filename = run_location+"/stats"+str(iex)+".txt"
uim.print_save_readout(print_list, stats_filename)

LMAX = 30
power_spectrum_unweighted = uhp.power_spectrum(imap_upgrade, LMAX)

fig = plt.figure()
ax = plt.axes()
plt.plot(np.arange(LMAX), power_spectrum_unweighted * 100.0)
ax.set_xticks(range(0, LMAX+1, int(LMAX/5)))
plt.xlim([0, LMAX])
plt.title("Intensity Modes")
plt.xlabel("l mode")
plt.ylabel(r"amplitude ($\%$)")
plt.savefig(run_location+"/hs_modes" + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

# Create time varying input deck

In [ ]:
import csv
import utils_intensity_map as uim
import netcdf_read_write as nrw
import utils_deck_generation as idg
import os

In [ ]:
load_from_file = True
iex = 0
run_dir = "Data"

num_examples = 0
sys_params = tdg.define_system_params(run_dir)

if load_from_file:
    dataset, dataset_params, deck_gen_params, facility_spec = idg.load_data_dicts_from_file(sys_params)
    
    cone_multiplier = np.zeros((dataset_params["num_profiles_per_config"], int(facility_spec['num_cones']/2)))
    for ind_profile in range(dataset_params["num_profiles_per_config"]):
        index_offset = ind_profile - dataset_params["num_profiles_per_config"]
        power_slice = slice(dataset_params["num_variables_per_beam"]+index_offset, dataset_params["num_input_params"], dataset_params["num_variables_per_beam"])
        cone_multiplier[ind_profile,:] = deck_gen_params["sim_params"][iex, power_slice]
    print(cone_multiplier)
    
else:
    dataset_params, facility_spec = tdg.define_dataset_params(int(num_examples), sys_params)
    cone_multiplier = np.zeros(facility_spec['num_cones']/2) + 1.0

num_cones = int(facility_spec['num_cones']/2) # Symmetry in top bottom hemisphere


## Read from "requested power" csv (per beam)

In [ ]:
file_dirname = "laser_pulse_data_pdd/N190204-003"
file_name = "N190204-003-999_LEH_requested_beam_powers"
file_type = ".csv"

num_lines = sum(1 for line in open(file_dirname + "/" + file_name + file_type))

beam_indices = [0] * facility_spec['nbeams']
beam_names = [0] * facility_spec['nbeams']
times = np.zeros((num_lines-1))
powers = np.zeros((facility_spec['nbeams'],num_lines-1))

total_power = 0

pulse_file = open(file_dirname + "/" + file_name + file_type)
pulse_data = csv.reader(pulse_file)

backlighter_beams = ("B161","B162","B163","B164","B415","B416","B417","B418")

irow=0
for row in pulse_data:
    if irow == 0:
        nbeams = len(row) - 1
        for ibeam in range(0,nbeams):
            #print(row[ibeam][2:])
            beam_names[ibeam] = "B"+row[ibeam+1][2:]
            beam_indices[ibeam] = np.where(facility_spec['Beam']==beam_names[ibeam])[0][0]
    else:
        times[irow-1] = row[0]
        powers[0:,irow-1] = row[1:]
    irow += 1
pulse_file.close()

backlighter_modified_beams = []
idealized_total_power_profile = 0
total_power_no_backlighter = 0

fig = plt.figure()
ax = plt.axes()
for ibeam in range(0,nbeams):
    if beam_names[ibeam] in backlighter_beams:
        print("Beam: " + beam_names[ibeam] + " is for the backlighter")
    else:
        total_power_no_backlighter += powers[ibeam,:]
        plt.plot(times, powers[ibeam,:])
    if np.max(powers[ibeam,:]) > 1.0:
        backlighter_modified_beams.append(beam_names[ibeam])
print(backlighter_modified_beams)
#plt.xlim([-0.5, 5.0])
plt.xlabel("Time (ns)")
plt.ylabel("Power per beam (TW)")
plt.savefig(file_dirname+'/laser_pulse_shape_per_beam' + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

total_power_with_backlighter = np.sum(powers, axis=0)
total_energy_with_backlighter = np.trapz(total_power_with_backlighter, x=times)
print("Total energy requested (with backlighter): {:.2f}".format(total_energy_with_backlighter), "kJ")
print("Max power (with backlighter): {:.2f}".format(np.max(total_power_with_backlighter)), "TW")

total_power = total_power_no_backlighter
total_energy = np.trapz(total_power_no_backlighter, x=times)
print("Total energy requested: {:.2f}".format(total_energy), "kJ")
print("Max power: {:.2f}".format(np.max(total_power)), "TW")

fig = plt.figure()
ax = plt.axes()
plt.plot(times, total_power_no_backlighter)
#plt.xlim([-0.5, 5.0])
plt.xlabel("Time (ns)")
plt.ylabel("Total Power (TW)")
plt.savefig(file_dirname+'/laser_pulse_shape_total' + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

idealized_power_profile = total_power / nbeams

fig = plt.figure()
ax = plt.axes()
plt.plot(times, idealized_power_profile)
#plt.xlim([-0.5, 5.0])
plt.xlabel("Time (ns)")
plt.ylabel("Idealized Power Profile per beam, no backlighter (TW)")
plt.savefig(file_dirname+'/idealized_laser_pulse_shape_per_beam' + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

times_old = times
powers_old = powers
total_power_old = total_power
times = np.zeros((num_cones,num_lines-1))
powers = np.zeros((num_cones,num_lines-1))
total_power = np.zeros((num_lines-1))

for icone in range(0,num_cones):
    powers[icone,:] = idealized_power_profile
    times[icone,:] = times_old
    total_power += powers[icone,:]

## Create simplified laser pulse for multi

In [ ]:
print(range(96,196,20))

In [ ]:
beam = facility_spec['Beam'][ibeam]
file_name = file_dirname + "/idealised_pulse_for_multi.txt"
string1 = ""
string2 = ""
with open(file_name,"w") as out:
    for icol in range(96,196,5):
        string1 = string1 + '{0:.2f},'.format(times[icol])
        string2 = string2 + '{0:.2f},'.format(total_power_no_backlighter[icol])
    out.write(string1)
    out.write('\r\n')
    out.write(string2)
out.close()

## Read from "experimentally delievered" csv N21 (per beam)

In [ ]:
file_dirname = "laser_pulse_data_pdd/N210519"
file_name = "N210519-001-999_LEH_reported_fixed"
file_type = ".csv"

num_lines = sum(1 for line in open(file_dirname + "/" + file_name + file_type))

nbeams = 188

beam_indices = [0] * nbeams
beam_names = [0] * nbeams
times = np.zeros((num_lines-1))
powers = np.zeros((nbeams,num_lines-1))

total_power = 0

pulse_file = open(file_dirname + "/" + file_name + file_type)
pulse_data = csv.reader(pulse_file)

backlighter_beams = ("B161","B162","B163","B164","B415","B416","B417","B418")

irow=0
for row in pulse_data:
    if irow == 0:
        nbeams = len(row) - 1
        for ibeam in range(0,nbeams):
            beam_names[ibeam] = "B"+row[ibeam+1][2:]
            beam_indices[ibeam] = np.where(facility_spec['Beam']==beam_names[ibeam])[0][0]
    else:
        times[irow-1] = row[0]
        powers[0:,irow-1] = row[1:]
    irow += 1
pulse_file.close()

backlighter_modified_beams = []
idealized_total_power_profile = 0
total_power_no_backlighter = 0

fig = plt.figure()
ax = plt.axes()
for ibeam in range(0,nbeams):
    if beam_names[ibeam] in backlighter_beams:
        print("Beam: " + beam_names[ibeam] + " is for the backlighter")
        #plt.plot(times, powers[ibeam,:])
    else:
        total_power_no_backlighter += powers[ibeam,:]
        plt.plot(times, powers[ibeam,:])
    if np.max(powers[ibeam,:]) > 1.0:
        backlighter_modified_beams.append(beam_names[ibeam])
print(backlighter_modified_beams)
#plt.xlim([-0.5, 11.0])
plt.xlabel("Time (ns)")
plt.ylabel("Power per beam (TW)")
plt.savefig(file_dirname+'/laser_pulse_shape_per_beam' + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

total_power_with_backlighter = np.sum(powers, axis=0)
total_energy_with_backlighter = np.trapz(total_power_with_backlighter, x=times)
print("Total energy requested (with backlighter): {:.2f}".format(total_energy_with_backlighter), "kJ")
print("Max power (with backlighter): {:.2f}".format(np.max(total_power_with_backlighter)), "TW")

total_power = total_power_no_backlighter
total_energy = np.trapz(total_power_no_backlighter, x=times)
print("Total energy requested: {:.2f}".format(total_energy), "kJ")
print("Max power: {:.2f}".format(np.max(total_power)), "TW")

fig = plt.figure()
ax = plt.axes()
plt.plot(times, total_power_no_backlighter)
#plt.xlim([-0.5, 11.0])
plt.xlabel("Time (ns)")
plt.ylabel("Total Power (TW)")
plt.savefig(file_dirname+'/laser_pulse_shape_total' + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

idealized_power_profile = total_power / nbeams

fig = plt.figure()
ax = plt.axes()
plt.plot(times, idealized_power_profile)
#plt.xlim([-0.5, 11.0])
plt.xlabel("Time (ns)")
plt.ylabel("Idealized Power Profile per beam, no backlighter (TW)")
plt.savefig(file_dirname+'/idealized_laser_pulse_shape_per_beam' + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

times_old = times
powers_old = powers
total_power_old = total_power
times = np.zeros((num_cones,num_lines-1))
powers = np.zeros((num_cones,num_lines-1))
total_power = np.zeros((num_lines-1))

for icone in range(0,num_cones):
    powers[icone,:] = idealized_power_profile
    times[icone,:] = times_old
    total_power += powers[icone,:]

## Read from "experimentally delievered" csv N19 (per beam)

In [ ]:
file_dirname = "laser_pulse_data_pdd/N190204-003"
file_name = "N190204-003-999_LEH_beam_powers"
file_type = ".csv"

num_lines = sum(1 for line in open(file_dirname + "/" + file_name + file_type))

beam_indices = [0] * facility_spec['nbeams']
beam_names = [0] * facility_spec['nbeams']
times = np.zeros((num_lines-1))
powers = np.zeros((facility_spec['nbeams'],num_lines-1))

total_power = 0

pulse_file = open(file_dirname + "/" + file_name + file_type)
pulse_data = csv.reader(pulse_file)

backlighter_beams = ("B161","B162","B163","B164","B415","B416","B417","B418")

irow=0
for row in pulse_data:
    if irow == 0:
        nbeams = len(row) - 1
        for ibeam in range(0,nbeams):
            beam_names[ibeam] = "B"+row[ibeam+1][2:]
            beam_indices[ibeam] = np.where(facility_spec['Beam']==beam_names[ibeam])[0][0]
    else:
        times[irow-1] = row[0]
        powers[0:,irow-1] = row[1:]
    irow += 1
pulse_file.close()

backlighter_modified_beams = []
idealized_total_power_profile = 0
total_power_no_backlighter = 0

fig = plt.figure()
ax = plt.axes()
for ibeam in range(0,nbeams):
    if beam_names[ibeam] in backlighter_beams:
        print("Beam: " + beam_names[ibeam] + " is for the backlighter")
    else:
        total_power_no_backlighter += powers[ibeam,:]
        plt.plot(times, powers[ibeam,:])
    if np.max(powers[ibeam,:]) > 1.0:
        backlighter_modified_beams.append(beam_names[ibeam])
print(backlighter_modified_beams)
#plt.xlim([-0.5, 5.0])
plt.xlabel("Time (ns)")
plt.ylabel("Power per beam (TW)")
plt.savefig(file_dirname+'/laser_pulse_shape_per_beam' + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

total_power_with_backlighter = np.sum(powers, axis=0)
total_energy_with_backlighter = np.trapz(total_power_with_backlighter, x=times)
print("Total energy requested (with backlighter): {:.2f}".format(total_energy_with_backlighter), "kJ")
print("Max power (with backlighter): {:.2f}".format(np.max(total_power_with_backlighter)), "TW")

total_power = total_power_no_backlighter
total_energy = np.trapz(total_power_no_backlighter, x=times)
print("Total energy requested: {:.2f}".format(total_energy), "kJ")
print("Max power: {:.2f}".format(np.max(total_power)), "TW")

fig = plt.figure()
ax = plt.axes()
plt.plot(times, total_power_no_backlighter)
#plt.xlim([-0.5, 5.0])
plt.xlabel("Time (ns)")
plt.ylabel("Total Power (TW)")
plt.savefig(file_dirname+'/laser_pulse_shape_total' + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

idealized_power_profile = total_power / nbeams

fig = plt.figure()
ax = plt.axes()
plt.plot(times, idealized_power_profile)
#plt.xlim([-0.5, 5.0])
plt.xlabel("Time (ns)")
plt.ylabel("Idealized Power Profile per beam, no backlighter (TW)")
plt.savefig(file_dirname+'/idealized_laser_pulse_shape_per_beam' + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

times_old = times
powers_old = powers
total_power_old = total_power
times = np.zeros((num_cones,num_lines-1))
powers = np.zeros((num_cones,num_lines-1))
total_power = np.zeros((num_lines-1))

for icone in range(0,num_cones):
    powers[icone,:] = idealized_power_profile
    times[icone,:] = times_old
    total_power += powers[icone,:]

## Read from "Ring" csv N21 (per beam)

In [ ]:
file_dirname = "laser_pulse_data_pdd/N210519"
file_name = "I_PDD_BePDD_DDShap_EEE_Ring"
file_type = ".csv"

num_lines = sum(1 for line in open(file_dirname + "/" + file_name + str(1) + file_type))

times = np.zeros((num_cones,num_lines))
powers = np.zeros((num_cones,num_lines))
fig = plt.figure()
ax = plt.axes()

total_power = 0
for icone in range(num_cones):
    pulse_file = open(file_dirname + "/" + file_name + str(icone+1) + file_type)
    pulse_data = csv.reader(pulse_file)
    i = 0
    for row in pulse_data:
        times[icone,i] = float(row[0])
        powers[icone,i] = float(row[1])
        i += 1

    pulse_file.close()
    
    total_power += powers[icone,:] * facility_spec["beams_per_cone"][icone] * 2
    
    plt.plot(times[icone,:], powers[icone,:], label=facility_spec['cone_names'][icone])
plt.xlabel("Time (ns)")
plt.ylabel("Power per beam (TW)")
plt.legend()
plt.savefig(file_dirname+'/laser_pulse_shape_per_beam' + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

total_energy = np.trapz(total_power, x=times[0,:])
print("Total energy requested: {:.2f}".format(total_energy), "kJ")
print("Max power: {:.2f}".format(np.max(total_power)), "TW")
#total_power = powers[0,:] * 32 + powers[1,:] * 32 + powers[2,:] * 64 + powers[3,:] * 64

fig = plt.figure()
ax = plt.axes()
plt.plot(times[0,:], total_power)
plt.xlabel("Time (ns)")
plt.ylabel("Total Power (TW)")
plt.savefig(file_dirname+'/laser_pulse_shape_total' + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

## Read from Olson paper data extraction (all beam pulse)

In [ ]:
file_dirname = "laser_pulse_data_pdd/ABLE_campaign"
file_name = "PDD_foam_target_olson_paper_pulse_data_extraction"
file_type = ".csv"


num_lines = sum(1 for line in open(file_dirname + "/" + file_name + file_type))

total_power_times = np.zeros((num_lines))
total_power = np.zeros((num_lines))

pulse_file = open(file_dirname + "/" + file_name + file_type)
pulse_data = csv.reader(pulse_file)
i = 0
for row in pulse_data:
    total_power_times[i] = float(row[0])
    total_power[i] = float(row[1])
    i += 1

pulse_file.close()

total_energy = np.trapz(total_power, x=total_power_times)
print("Total energy requested: {:.2f}".format(total_energy), "kJ")
#total_power = powers[0,:] * 32 + powers[1,:] * 32 + powers[2,:] * 64 + powers[3,:] * 64

fig = plt.figure()
ax = plt.axes()
plt.plot(total_power_times, total_power)
plt.xlabel("Time (ns)")
plt.ylabel("Total Power (TW)")
plt.savefig(file_dirname+'/laser_pulse_shape_total' + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

times = np.zeros((num_cones,num_lines))
powers = np.zeros((num_cones,num_lines))
for icone in range(num_cones):
    times[icone,:] = total_power_times
    powers[icone,:] = total_power / facility_spec["nbeams"]

## Output Olsen laser pulse for MULTI

In [ ]:
times = np.zeros((num_cones,num_lines))
powers = np.zeros((num_cones,num_lines))
for icone in range(num_cones):
    times[icone,:] = total_power_times
    powers[icone,:] = total_power / facility_spec["nbeams"]

# Add extra 8% power for 23.5 cones
powers[0,:] = powers[0,:] * 1.08
renorm_fac = (facility_spec["nbeams"] - facility_spec["beams_per_cone"][0] * 2 
              + facility_spec["beams_per_cone"][0] * 2 * 1.08) / 192
powers = powers / renorm_fac

fig = plt.figure()
ax = plt.axes()
total_energy = 0.0
for icone in range(num_cones):
    plt.plot(times[icone,:], powers[icone,:], label=facility_spec['cone_names'][icone])
    cone_powers = powers[icone,:] * facility_spec["beams_per_cone"][icone] * 2
    cone_energy = np.trapz(cone_powers, x=times[icone,:])
    print(cone_energy)
    total_energy += cone_energy
print(total_energy)
plt.xlabel("Time (ns)")
plt.ylabel("Power per beam (TW)")
plt.legend()
plt.savefig(file_dirname+'/laser_pulse_shape_per_beam' + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

## Output pulse shapes unmodified

In [ ]:


for ibeam in range(facility_spec['nbeams']):
    beam = facility_spec['Beam'][ibeam]
    file_name = file_dirname + "/pulse_" + beam + ".txt"
    
    cone_name = facility_spec["Cone"][ibeam]
    if (cone_name == 23.5):
        icone = 0
    elif (cone_name == 30):
        icone = 1
    elif (cone_name == 44.5):
        icone = 2
    else:
        icone = 3
    
    with open(file_name,"w") as out:
        for irow in range(len(times[0,:])):
            out.write(str(times[icone,irow]) + ' ' + str(powers[icone,irow]))
            out.write('\r\n')
    out.close()


## Output pulse shapes with cone power fractions (time varying)

In [ ]:
#print(facility_spec['beams_per_cone'])
#for icone in range(facility_spec['num_cones']):

In [ ]:
cone_power_multipliers_times = np.array((5.0, 9.0)) #np.array((1.0, 4.5)) #(0.0, 4.0) #for N19

normalized_power = total_power / np.max(total_power)
num_times = np.shape(times)[1]

powers_reweighted = np.zeros((np.shape(times))) + 1.0
cone_time_varying_weighting = np.zeros(np.shape(times))
print(np.shape(cone_time_varying_weighting))
total_energy_unweighted = 0.

fig = plt.figure()
ax = plt.axes()
fig2 = plt.figure()
ax2 = plt.axes()
for icone in range(num_cones):
    for tind in range(num_times):
        #tind = 150
        t0 = times[icone, tind]
        if t0 <= cone_power_multipliers_times[0]:
            cone_time_varying_weighting[icone,tind] = cone_multiplier[0,icone]

        elif t0 >= cone_power_multipliers_times[-1]:
            cone_time_varying_weighting[icone,tind] = cone_multiplier[-1,icone]

        else:
            ind_min = np.where(cone_power_multipliers_times < t0)[0][-1]
            ind_max = ind_min + 1

            diff_min_max = cone_power_multipliers_times[ind_max] - cone_power_multipliers_times[ind_min]

            diff_min = (t0 - cone_power_multipliers_times[ind_min]) / diff_min_max
            diff_max = (cone_power_multipliers_times[ind_max] - t0) / diff_min_max
            cone_time_varying_weighting[icone,tind] = cone_multiplier[ind_min,icone] * diff_max + cone_multiplier[ind_max,icone] * diff_min
            #print(cone_multiplier[ind_min,icone], diff_min)
            #print(cone_multiplier[ind_max,icone], diff_max)
        #print(times[icone,tind], cone_time_varying_weighting[icone,tind])

    powers_reweighted = normalized_power * cone_time_varying_weighting
    ax.plot(times[icone,:], cone_time_varying_weighting[icone,:])
    ax2.plot(times[icone,:], powers_reweighted[icone,:])
    
    # This section rescales the pulse to achieve same total 
    #total_energy_unweighted += np.trapz(powers_reweighted[icone,:], x=times[icone,:]) * facility_spec["beams_per_cone"][icone] * 2
    
#renorm_factor = total_energy_unweighted / total_energy
#powers_reweighted = powers_reweighted / renorm_factor
#print("Total energy: ", total_energy, "unweighted energy: ", total_energy_unweighted, "normalisation factor: ", renorm_factor)
    
#ax.set_xlim([-0.5, 8.0])
#ax2.set_xlim([-0.5, 8.0])
ax.set_xlabel("Time (ns)")
ax.set_ylabel("Weighting")
ax2.set_xlabel("Time (ns)")
ax2.set_ylabel("time varying per beam power (TW)")




In [ ]:
total_power_reweighted = 0.0

print(file_dirname)
for ibeam in range(facility_spec['nbeams']):
    beam = facility_spec['Beam'][ibeam]
    file_name = file_dirname + "/pulse_" + beam + ".txt"
    
    cone_name = facility_spec["Cone"][ibeam]
    if (cone_name == 23.5):
        icone = 0
    elif (cone_name == 30):
        icone = 1
    elif (cone_name == 44.5):
        icone = 2
    else:
        icone = 3
    total_power_reweighted += powers_reweighted[icone,:]
    
    with open(file_name,"w") as out:
        for irow in range(len(times[0,:])):
            out.write(str(times[icone,irow]) + ' ' + str(powers_reweighted[icone,irow]))
            out.write('\r\n')
    out.close()

print(np.max(powers_reweighted, axis=1))
fig = plt.figure()
ax = plt.axes()
for icone in range(num_cones):
    plt.plot(times[icone,:], powers_reweighted[icone,:], label=facility_spec['cone_names'][icone])
plt.xlabel("Time (ns)")
plt.ylabel("Power per beam (TW)")
#plt.xlim([-0.5, 10.0])
plt.legend()
plt.savefig(file_dirname+'/new_laser_pulse_shape_per_beam' + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

total_energy_reweighted = np.trapz(total_power_reweighted, x=times[0,:])
print("Total energy requested: {:.2f}".format(total_energy_reweighted), "kJ")
#total_power = powers[0,:] * 32 + powers[1,:] * 32 + powers[2,:] * 64 + powers[3,:] * 64
print("Max power: {:.2f}".format(np.max(total_power_reweighted)), "TW")

fig = plt.figure()
ax = plt.axes()
plt.plot(times[0,:], total_power_reweighted)
plt.xlabel("Time (ns)")
plt.ylabel("Total Power (TW)")
#plt.xlim([-0.5, 10.0])
plt.savefig(file_dirname+'/new_laser_pulse_shape_total' + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

In [ ]:
scaling = total_energy / total_energy_reweighted
print(scaling)

## Output pulse shapes with cone power fractions

In [ ]:
normalized_power = total_power / np.max(total_power)
total_power_reweighted = 0.0
powers_reweighted = np.zeros((np.shape(times)))

for ibeam in range(facility_spec['nbeams']):
    beam = facility_spec['Beam'][ibeam]
    file_name = file_dirname + "/pulse_" + beam + ".txt"
    
    cone_name = facility_spec["Cone"][ibeam]
    if (cone_name == 23.5):
        icone = 0
    elif (cone_name == 30):
        icone = 1
    elif (cone_name == 44.5):
        icone = 2
    else:
        icone = 3
    powers_reweighted[icone,:] = normalized_power[:] * cone_multiplier[icone]
    total_power_reweighted += powers_reweighted[icone,:]
    
    with open(file_name,"w") as out:
        for irow in range(len(times[0,:])):
            out.write(str(times[icone,irow]) + ' ' + str(powers_reweighted[icone,irow]))
            out.write('\r\n')
    out.close()

print(np.max(powers_reweighted, axis=1))
fig = plt.figure()
ax = plt.axes()
for icone in range(num_cones):
    plt.plot(times[icone,:], powers_reweighted[icone,:], label=facility_spec['cone_names'][icone])
plt.xlabel("Time (ns)")
plt.ylabel("Power per beam (TW)")
#plt.xlim([-0.5, 10.0])
plt.legend()
plt.savefig(file_dirname+'/new_laser_pulse_shape_per_beam' + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

total_energy_reweighted = np.trapz(total_power_reweighted, x=times[0,:])
print("Total energy requested: {:.2f}".format(total_energy_reweighted), "kJ")
#total_power = powers[0,:] * 32 + powers[1,:] * 32 + powers[2,:] * 64 + powers[3,:] * 64
print("Max power: {:.2f}".format(np.max(total_power_reweighted)), "TW")

fig = plt.figure()
ax = plt.axes()
plt.plot(times[0,:], total_power_reweighted)
plt.xlabel("Time (ns)")
plt.ylabel("Total Power (TW)")
#plt.xlim([-0.5, 10.0])
plt.savefig(file_dirname+'/new_laser_pulse_shape_total' + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')


## Rescale powers for same total energy

In [ ]:
scaling = total_energy / total_energy_reweighted
print(scaling)

normalized_power = total_power / np.max(total_power)
total_power_reweighted = 0.0
#powers_reweighted = np.zeros((np.shape(times)))

print(file_dirname)
for ibeam in range(facility_spec['nbeams']):
    beam = facility_spec['Beam'][ibeam]
    file_name = file_dirname + "/pulse_" + beam + ".txt"
    
    cone_name = facility_spec["Cone"][ibeam]
    if (cone_name == 23.5):
        icone = 0
    elif (cone_name == 30):
        icone = 1
    elif (cone_name == 44.5):
        icone = 2
    else:
        icone = 3
    total_power_reweighted += powers_reweighted[icone,:] * scaling
    
    with open(file_name,"w") as out:
        for irow in range(len(times[0,:])):
            out.write(str(times[icone,irow]) + ' ' + str(powers_reweighted[icone,irow] * scaling))
            out.write('\r\n')
    out.close()

print(np.max(powers_reweighted * scaling, axis=1))
fig = plt.figure()
ax = plt.axes()
for icone in range(num_cones):
    plt.plot(times[icone,:], powers_reweighted[icone,:] * scaling, label=facility_spec['cone_names'][icone])
plt.xlabel("Time (ns)")
plt.ylabel("Power per beam (TW)")
plt.legend()
#plt.xlim([-0.5, 8.0])
plt.savefig(file_dirname+'/new_scaled_laser_pulse_shape_per_beam' + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

total_energy_reweighted = np.trapz(total_power_reweighted, x=times[0,:])
print("Total energy requested: {:.2f}".format(total_energy_reweighted), "kJ")
#total_power = powers[0,:] * 32 + powers[1,:] * 32 + powers[2,:] * 64 + powers[3,:] * 64
print("Max power: {:.2f}".format(np.max(total_power_reweighted)), "TW")

fig = plt.figure()
ax = plt.axes()
plt.plot(times[0,:], total_power_reweighted)
plt.xlabel("Time (ns)")
plt.ylabel("Total Power (TW)")
#plt.xlim([-0.5, 8.0])
plt.savefig(file_dirname+'/new_scaled_laser_pulse_shape_total' + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')


## Output decks with beam pointings

In [ ]:
#sys_params = tdg.define_system_params(file_dirname)
#sys_params["run_clean"] = False
laser_init_time = 0.0
print("Lasers set to initialize at time: {:.2f}ns".format(laser_init_time))
deck_gen_params['t0'] = laser_init_time

#dataset_params = idg.create_run_files(dataset_params, sys_params, facility_spec)

#idg.generate_input_deck(facility_spec, file_dirname)

if os.path.exists(file_dirname + "/ifriit_inputs.txt"):
    os.remove(file_dirname + "/ifriit_inputs.txt")
idg.generate_input_pointing_and_pulses(iex, _, facility_spec, deck_gen_params, file_dirname, dataset_params["run_type"])

# Beam Spot

In [ ]:
def nif_beam_spot(x, y, sup_gauss_dict):
    
    c0 = sup_gauss_dict["c0"]
    c1 = sup_gauss_dict["c1"]
    c2 = sup_gauss_dict["c2"]
    c3 = sup_gauss_dict["c3"]
    intensity = c0 * np.exp(-((x/c1)**2+(y/c2)**2)**(c3/2))
    
    return intensity

In [ ]:
def nif_beam_spot_FWHM(x, y, sup_gauss_dict):
    
    c0 = sup_gauss_dict["c0"]
    FWHMx = sup_gauss_dict["FWHMx"]
    FWHMy = sup_gauss_dict["FWHMy"]
    c3 = sup_gauss_dict["c3"]
    c5 = 2.0**c3 * np.log(2.0)
    intensity = c0 * np.exp(-c5*((x/FWHMx)**2+(y/FWHMy)**2)**(c3/2))
    
    return intensity

In [ ]:
def nif_beam_spot_FWHM_mike(x, y, sup_gauss_dict):
    
    c0 = sup_gauss_dict["c0"]
    FWHMx = sup_gauss_dict["FWHMx"]
    FWHMy = sup_gauss_dict["FWHMy"]
    c3 = sup_gauss_dict["c3"]
    c5 = 2.0 * np.log(2.0)**c3
    intensity = np.exp(-((-x/FWHMx)**2+(-y/FWHMy)**2)**(c3/2))
    
    return intensity

In [ ]:
def nif_beam_spot_FWHM_xavier(x, y, sup_gauss_dict):
    
    c0 = sup_gauss_dict["c0"]
    FWHMx = sup_gauss_dict["FWHMx"]
    FWHMy = sup_gauss_dict["FWHMy"]
    c3x = sup_gauss_dict["c3x"]
    c3y = sup_gauss_dict["c3y"]
    c5 = 2.0 * np.log(2.0)**c3
    
    dr = np.sqrt(x**2 + y**2)
    alpha = np.sqrt((c3x * x / dr)**2 + (c3y * y / dr)**2)
    intensity = np.exp(-((-x / FWHMx)**2 + (-y / FWHMy)**2)**(alpha / 2))
    
    return intensity

In [ ]:
def super_gaussian_shape(c0, c1, c2, c3):
    
    sup_gauss_dict = {}
    sup_gauss_dict["c0"] = c0
    sup_gauss_dict["c1"] = c1
    sup_gauss_dict["c2"] = c2
    sup_gauss_dict["c3"] = c3
    
    return sup_gauss_dict

In [ ]:
run_dir = "Data"
num_examples = 1
sys_params = tdg.define_system_params(run_dir)
dataset_params, facility_spec = tdg.define_dataset_params(int(num_examples), sys_params)
num_cones = int(facility_spec['num_cones']/2) # Symmetry in top bottom hemisphere

## All NIF CPP

In [ ]:
mesh_lim = 1.5
num_points = 1000
x = np.linspace(-mesh_lim, mesh_lim, num_points)
y = np.linspace(-mesh_lim, mesh_lim, num_points)
X, Y = np.meshgrid(x,y)

c0 = [0.55273, 0.62017, 1.30020, 1.46540]
c1 = [0.93836, 0.89096, 0.67533, 0.65491]
c2 = [0.66859, 0.63113, 0.40627, 0.37022]
c3 = [8.7514,  7.8105,  4.3210,  4.3427]

r_a = [0.882, 0.824, 0.635, 0.593]
r_b = [0.631, 0.590, 0.367, 0.343]

fig = plt.figure()
for icone in range(0, num_cones):
    sup_gauss_dict = super_gaussian_shape(c0[icone], c1[icone], c2[icone], c3[icone])
    intensity = nif_beam_spot(X, Y, sup_gauss_dict)
    fig.add_subplot(1, 4, icone+1)
    plt.pcolormesh(X, Y, intensity)
    plt.gca().set_aspect('equal', adjustable='box')
    
    # Tom Chapman uses a r_a threshold in his data
    #intensity_threshold = nif_beam_spot(r_a[icone], 0.0, sup_gauss_dict)
    # This is a more correct way to do the Chapman threshold
    #intensity_threshold = nif_beam_spot(r_a[icone]/np.sqrt(2), r_b[icone]/np.sqrt(2), sup_gauss_dict)
    # the 1/e
    intensity_threshold = nif_beam_spot(c1[icone]/np.sqrt(2), c2[icone]/np.sqrt(2), sup_gauss_dict)
    
    print(intensity_threshold, np.max(intensity), c0[icone], np.exp(-1) * c0[icone])

    inside_indices = np.where(intensity > intensity_threshold)
    
    power_inside = np.sum(intensity[inside_indices])
    power_total = np.sum(intensity)
    power_fraction = power_inside / power_total
    print("Ring ", icone)
    print("Power fraction enclosed by threshold {:.3f}, ".format(power_fraction))
    print("Area of within threshold {:.3f}, ".format(np.shape(inside_indices)[1] * (2.0 * mesh_lim / num_points)**2))
    print("Area of a.b ellipse {:.3f}, ".format(np.pi * r_a[icone] * r_b[icone]))
    print("Area of c1.c2 ellipse {:.3f}, ".format(np.pi * c1[icone] * c2[icone]))


In [ ]:
FWHMx = [1.800, 1.700, 1.241, 1.204]
FWHMy = [1.282, 1.204, 0.7465, 0.6805]

fig = plt.figure()
for icone in range(0, num_cones):
    sup_gauss_dict = super_gaussian_shape(c0[icone], c1[icone], c2[icone], c3[icone])
    sup_gauss_dict["FWHMx"] = FWHMx[icone]
    sup_gauss_dict["FWHMy"] = FWHMy[icone]
    intensity = nif_beam_spot_FWHM(X, Y, sup_gauss_dict)
    
    fig.add_subplot(1, 4, icone+1)
    plt.pcolormesh(X, Y, intensity)
    plt.gca().set_aspect('equal', adjustable='box')
    
    #intensity_threshold = nif_beam_spot_FWHM(FWHMx[icone]/2.0, 0.0, sup_gauss_dict)
    intensity_threshold = nif_beam_spot_FWHM(FWHMx[icone]/2.0/np.sqrt(2), FWHMy[icone]/2.0/np.sqrt(2), sup_gauss_dict)
    #intensity_threshold = nif_beam_spot_FWHM(2.0*FWHMx[icone]*np.log(2.0)**(1.0/c3[icone]), 0.0, sup_gauss_dict)
    inside_indices = np.where(intensity > intensity_threshold)
    
    power_inside = np.sum(intensity[inside_indices])
    power_total = np.sum(intensity)
    power_fraction = power_inside / power_total
    print("Ring ", icone)
    print("Power fraction enclosed by threshold {:.3f}, ".format(power_fraction))
    print("Area of within threshold {:.3f}, ".format(np.shape(inside_indices)[1] * (2.0 * mesh_lim / num_points)**2))
    print("Area of FWHMx/2.FWHMy/2 ellipse {:.3f}, ".format(np.pi * FWHMx[icone]/2.0 * FWHMy[icone]/2.0))
    

## OMEGA

In [ ]:
fig, ax = plt.subplots(figsize=(6, 3), dpi=200)

boundaries_mm = 0.6
x = np.linspace(-boundaries_mm,boundaries_mm,1000)
y = np.linspace(-boundaries_mm,boundaries_mm,1000)

c0 = 1.0
c1 = 0.358
c2 = 0.358
c3 = 5.2

sup_gauss_dict = super_gaussian_shape(c0, c1, c2, c3)
X, Y = np.meshgrid(x,y)
intensity = nif_beam_spot(x, 0.0, sup_gauss_dict) #/ sum_beamspot
ax.plot(x,intensity, linewidth=3, color="tab:orange") #/np.max(intensity)

ax.set_xlabel("Radius (mm)")
ax.set_ylabel("Power (norm)")
#ax.set_ylim([0, 1.0])
plt.savefig(sys_params["root_dir"]+"/"+sys_params["figure_location"]+"/beamspot_sg5_zoom_normalised" + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

intensity = nif_beam_spot(X, Y, sup_gauss_dict)
plt.figure()
plt.pcolormesh(X, Y, intensity)
plt.gca().set_aspect('equal', adjustable='box')

intensity_threshold = nif_beam_spot(c1, 0.0, sup_gauss_dict)
intensity_threshold = nif_beam_spot(c1/np.sqrt(2), c2/np.sqrt(2), sup_gauss_dict)
#intensity_threshold = nif_beam_spot(r_a, 0.0, sup_gauss_dict)
print(intensity_threshold)
print(nif_beam_spot(0.0, c2, sup_gauss_dict))

inside_indices = np.where(intensity > intensity_threshold)
print(np.shape(inside_indices))
power_inside = np.sum(intensity[inside_indices])
power_total = np.sum(intensity)
power_fraction = power_inside / power_total
print(power_fraction)

intensity[inside_indices] = 0.0
plt.figure()
plt.pcolormesh(X, Y, intensity)
plt.gca().set_aspect('equal', adjustable='box')


## Custom

In [ ]:
fig, ax = plt.subplots(figsize=(6, 3), dpi=200)

boundaries_mm = 2.5
x = np.linspace(-boundaries_mm,boundaries_mm,1000)
y = np.linspace(-boundaries_mm,boundaries_mm,1000)

c0 = 1.0
c1 = 1.7
c2 = 1.7
c3 = 5.2

sup_gauss_dict = super_gaussian_shape(c0, c1, c2, c3)
"""
X, Y = np.meshgrid(x,y)
intensity = nif_beam_spot(X, Y, sup_gauss_dict)
sum_beamspot = np.sum(intensity) / 80000
c0 = 1/sum_beamspot#0.55273
sup_gauss_dict = super_gaussian_shape(c0, c1, c2, c3)
intensity = nif_beam_spot(X, Y, sup_gauss_dict)
sum_beamspot = np.sum(intensity) / 80000
print(sum_beamspot)
"""
intensity = nif_beam_spot(x, 0.0, sup_gauss_dict) #/ sum_beamspot
ax.plot(x,intensity, linewidth=3, color="tab:orange") #/np.max(intensity)
#################################

"""
x = np.linspace(-boundaries_mm,boundaries_mm,1000)
y = np.linspace(-boundaries_mm,boundaries_mm,1000)
X, Y = np.meshgrid(x,y)

c0 = 0.55273
c1 = 1.20
c2 = 1.20
c3 = 5.2

sup_gauss_dict = super_gaussian_shape(c0, c1, c2, c3)
intensity = nif_beam_spot(x, 0.0, sup_gauss_dict)
ax.plot(x,intensity/np.max(intensity), linewidth=3, color="tab:orange")
"""
#################################

x = np.linspace(-boundaries_mm,boundaries_mm,1000)
y = np.linspace(-boundaries_mm,boundaries_mm,1000)
c0 = 1.0#0.55273
c1 = 1.043#0.7
c2 = 1.043#0.7
c3 = 2.33#5.2
sup_gauss_dict = super_gaussian_shape(c0, c1, c2, c3)
X, Y = np.meshgrid(x,y)
"""
intensity = nif_beam_spot(X, Y, sup_gauss_dict)
sum_beamspot = np.sum(intensity) / 80000
c0 = 1/sum_beamspot#0.55273
sup_gauss_dict = super_gaussian_shape(c0, c1, c2, c3)
intensity = nif_beam_spot(X, Y, sup_gauss_dict)
sum_beamspot = np.sum(intensity) / 80000
print(sum_beamspot)
"""
intensity = nif_beam_spot(x, 0.0, sup_gauss_dict)
ax.plot(x,intensity, linewidth=3, color="tab:purple") #/np.max(intensity)
#################################

x = np.linspace(-boundaries_mm,boundaries_mm,1000)
y = np.linspace(-boundaries_mm,boundaries_mm,1000)

c0 = 1.0#0.55273
c1 = 0.576#0.7
c2 = 0.576#0.7
c3 = 6.0#5.2

sup_gauss_dict = super_gaussian_shape(c0, c1, c2, c3)
X, Y = np.meshgrid(x,y)
"""
intensity = nif_beam_spot(X, Y, sup_gauss_dict)
sum_beamspot = np.sum(intensity) / 80000
c0 = 1/sum_beamspot#0.55273
sup_gauss_dict = super_gaussian_shape(c0, c1, c2, c3)
intensity = nif_beam_spot(X, Y, sup_gauss_dict)
sum_beamspot = np.sum(intensity) / 80000
print(sum_beamspot)
"""
intensity = nif_beam_spot(x, 0.0, sup_gauss_dict) #/ sum_beamspot
ax.plot(x,intensity, linewidth=3, color="tab:green")#/np.max(intensity)

#################################
ax.set_xlabel("Radius (mm)")
ax.set_ylabel("Power (norm)")
#ax.set_ylim([0, 1.0])
plt.savefig(sys_params["root_dir"]+"/"+sys_params["figure_location"]+"/beamspot_sg5_zoom_normalised" + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

intensity = nif_beam_spot(X, Y, sup_gauss_dict)
plt.figure()
plt.pcolormesh(X, Y, intensity)
plt.gca().set_aspect('equal', adjustable='box')

intensity_threshold = nif_beam_spot(c1, 0.0, sup_gauss_dict)
intensity_threshold = nif_beam_spot(c1/np.sqrt(2), c2/np.sqrt(2), sup_gauss_dict)
#intensity_threshold = nif_beam_spot(r_a, 0.0, sup_gauss_dict)
print(intensity_threshold)
print(nif_beam_spot(0.0, c2, sup_gauss_dict))

inside_indices = np.where(intensity > intensity_threshold)
print(np.shape(inside_indices))
power_inside = np.sum(intensity[inside_indices])
power_total = np.sum(intensity)
power_fraction = power_inside / power_total
print(power_fraction)

intensity[inside_indices] = 0.0
plt.figure()
plt.pcolormesh(X, Y, intensity)
plt.gca().set_aspect('equal', adjustable='box')


## 23.5 deg CPP 

In [ ]:

x = np.linspace(-2,2,1000)
y = np.linspace(-2,2,1000)
X, Y = np.meshgrid(x,y)

c0 = 0.55273
c1 = 0.93836
c2 = 0.66859
c3 = 8.7514

r_a = 0.882
r_b = 0.631

sup_gauss_dict = super_gaussian_shape(c0, c1, c2, c3)
intensity = nif_beam_spot(x, 0.0, sup_gauss_dict)
plt.figure()
plt.plot(x,intensity)

intensity = nif_beam_spot(X, Y, sup_gauss_dict)
plt.figure()
plt.pcolormesh(X, Y, intensity)
plt.gca().set_aspect('equal', adjustable='box')

intensity_threshold = nif_beam_spot(c1, 0.0, sup_gauss_dict)
intensity_threshold = nif_beam_spot(c1/np.sqrt(2), c2/np.sqrt(2), sup_gauss_dict)
intensity_threshold = nif_beam_spot(r_a, 0.0, sup_gauss_dict)
print(intensity_threshold)
print(nif_beam_spot(0.0, c2, sup_gauss_dict))

inside_indices = np.where(intensity > intensity_threshold)
print(np.shape(inside_indices))
power_inside = np.sum(intensity[inside_indices])
power_total = np.sum(intensity)
power_fraction = power_inside / power_total
print(power_fraction)

intensity[inside_indices] = 0.0
plt.figure()
plt.pcolormesh(X, Y, intensity)
plt.gca().set_aspect('equal', adjustable='box')


## LMJ-A CPP 

In [ ]:
mesh_lim = 1.0
num_points = 1000
x = np.linspace(-mesh_lim, mesh_lim, num_points)
y = np.linspace(-mesh_lim, mesh_lim, num_points)
X, Y = np.meshgrid(x,y)

c0 = np.array([1.0])     # Guess
c1 = np.array([0.435]) # Guess
c2 = np.array([0.225]) # Guess
c3 = np.array([2.4])  # Guess

r_a = np.array([0.435]) # Guess
r_b = np.array([0.225])  # Guess

icone = 0

sup_gauss_dict = super_gaussian_shape(c0, c1, c2, c3)
intensity = nif_beam_spot(x, 0.0, sup_gauss_dict)
plt.figure()
plt.plot(x,intensity)
plt.xlabel("Width (mm)")
plt.ylabel("Intensity ($1/I_{max}$)")
plt.savefig('LMJ_spot_type_A_cross_section', dpi=300, bbox_inches='tight')
intensity_old_def = intensity

intensity = nif_beam_spot(X, Y, sup_gauss_dict)
plt.figure()
plt.pcolormesh(X, Y, intensity)
plt.xlabel("Width (mm)")
plt.ylabel("Length (mm)")
plt.gca().set_aspect('equal', adjustable='box')
plt.savefig('LMJ_spot_type_A_spot', dpi=300, bbox_inches='tight')

sup_gauss_dict = super_gaussian_shape(c0[icone], c1[icone], c2[icone], c3[icone])
intensity = nif_beam_spot(X, Y, sup_gauss_dict)
    
intensity_threshold = nif_beam_spot(c1[icone]/np.sqrt(2), c2[icone]/np.sqrt(2), sup_gauss_dict)
    
print(intensity_threshold, np.max(intensity), c0[icone], np.exp(-1) * c0[icone])
intensity_threshold = 0.13 * np.max(intensity)

inside_indices = np.where(intensity > intensity_threshold)
    
power_inside = np.sum(intensity[inside_indices])
power_total = np.sum(intensity)
power_fraction = power_inside / power_total
print("Power fraction enclosed by threshold {:.3f}, ".format(power_fraction))
print("Area of within threshold {:.3f}, ".format(np.shape(inside_indices)[1] * (2.0 * mesh_lim / num_points)**2))
print("Area of a.b ellipse {:.3f}, ".format(np.pi * r_a[icone] * r_b[icone]))
print("Area of c1.c2 ellipse {:.3f}, ".format(np.pi * c1[icone] * c2[icone]))


In [ ]:
# Mike Belyaev's method
icone = 0
c0 = np.array([1.0])     # Guess
c1 = np.array([0.435]) # Guess
c2 = np.array([0.225]) # Guess
c3 = np.array([2.7])

FWHMx = [0.435]
FWHMy = [0.225]
print(FWHMx[0] / np.cos(59*np.pi/180.0)) # Circular spot at 59degrees from LEH

sup_gauss_dict = super_gaussian_shape(c0[icone], c1[icone], c2[icone], c3[icone])
sup_gauss_dict["FWHMx"] = FWHMx[icone]
sup_gauss_dict["FWHMy"] = FWHMy[icone]

mesh_lim = 1.1
num_points = 1000
x = np.linspace(-mesh_lim, mesh_lim, num_points)
y = np.linspace(-mesh_lim, mesh_lim, num_points)
X, Y = np.meshgrid(x,y)

intensity = nif_beam_spot_FWHM_mike(x, 0.0, sup_gauss_dict)
plt.figure()
plt.plot(x,intensity)
plt.xlabel("Width (mm)")
plt.ylabel("Intensity ($1/I_{max}$)")
plt.savefig('LMJ_spot_type_A_cross_section', dpi=300, bbox_inches='tight')
intensity_single_order_x = intensity

intensity = nif_beam_spot_FWHM_mike(0.0, y, sup_gauss_dict)
plt.figure()
plt.plot(x,intensity)
plt.xlabel("Width (mm)")
plt.ylabel("Intensity ($1/I_{max}$)")
plt.savefig('LMJ_spot_type_A_cross_section', dpi=300, bbox_inches='tight')
intensity_single_order_y = intensity

intensity = nif_beam_spot_FWHM_mike(X, Y, sup_gauss_dict)
plt.figure()
plt.pcolormesh(X, Y, intensity)
plt.xlabel("Width (mm)")
plt.ylabel("Length (mm)")
plt.gca().set_aspect('equal', adjustable='box')
plt.savefig('LMJ_spot_type_A_spot', dpi=300, bbox_inches='tight')

In [ ]:
# Xavier's method
icone = 0
c0 = np.array([1.0])     # Guess
c1 = np.array([0.435]) # Guess
c2 = np.array([0.225]) # Guess
c3 = np.array([0.0])
c3x = np.array([3.0])
c3y = np.array([2.4])

FWHMx = [0.435]
FWHMy = [0.225]
print(FWHMx[0] / np.cos(59*np.pi/180.0)) # Circular spot at 59degrees from LEH

sup_gauss_dict = super_gaussian_shape(c0[icone], c1[icone], c2[icone], c3[icone])
sup_gauss_dict["FWHMx"] = FWHMx[icone]
sup_gauss_dict["FWHMy"] = FWHMy[icone]
sup_gauss_dict["c3x"] = c3x[icone]
sup_gauss_dict["c3y"] = c3y[icone]

mesh_lim = 1.0
num_points = 1000
x = np.linspace(-mesh_lim, mesh_lim, num_points)
y = np.linspace(-mesh_lim, mesh_lim, num_points)
X, Y = np.meshgrid(x,y)

intensity = nif_beam_spot_FWHM_xavier(x, 0.0, sup_gauss_dict)
plt.figure()
plt.plot(x,intensity)
plt.plot(x,intensity_single_order_x)
plt.xlabel("Width (mm)")
plt.ylabel("Intensity ($1/I_{max}$)")
plt.savefig('LMJ_spot_type_A_cross_section', dpi=300, bbox_inches='tight')

intensity = nif_beam_spot_FWHM_xavier(0.0, y, sup_gauss_dict)
plt.figure()
plt.plot(x,intensity)
plt.plot(x,intensity_single_order_y)
plt.xlabel("Width (mm)")
plt.ylabel("Intensity ($1/I_{max}$)")
plt.savefig('LMJ_spot_type_A_cross_section', dpi=300, bbox_inches='tight')

intensity = nif_beam_spot_FWHM_xavier(X, Y, sup_gauss_dict)
plt.figure()
plt.pcolormesh(X, Y, intensity)
plt.xlabel("Width (mm)")
plt.ylabel("Length (mm)")
plt.gca().set_aspect('equal', adjustable='box')
plt.savefig('LMJ_spot_type_A_spot', dpi=300, bbox_inches='tight')

## LMJ-A CPP, quadsplitting

In [ ]:
x_offset = 0.31
y_offset = 0.16

mesh_lim = 2.0
num_points = 1000
x = np.linspace(-mesh_lim, mesh_lim, num_points)
y = np.linspace(-mesh_lim, mesh_lim, num_points)
X, Y = np.meshgrid(x,y)

# Xavier's method
icone = 0
c0 = np.array([1.0])     # Guess
c1 = np.array([0.435]) # Guess
c2 = np.array([0.225]) # Guess
c3 = np.array([2.7])

FWHMx = [0.435]
FWHMy = [0.225]
print(FWHMx[0] / np.cos(59*np.pi/180.0)) # Circular spot at 59degrees from LEH

sup_gauss_dict = super_gaussian_shape(c0[icone], c1[icone], c2[icone], c3[icone])
sup_gauss_dict["FWHMx"] = FWHMx[icone]
sup_gauss_dict["FWHMy"] = FWHMy[icone]

intensity_sup_pos1 = nif_beam_spot_FWHM_mike(x+x_offset, 0.0-y_offset, sup_gauss_dict)
intensity_sup_pos2 = nif_beam_spot_FWHM_mike(x-x_offset, 0.0-y_offset, sup_gauss_dict)
intensity_sup_pos3 = nif_beam_spot_FWHM_mike(x+x_offset, 0.0+y_offset, sup_gauss_dict)
intensity_sup_pos4 = nif_beam_spot_FWHM_mike(x-x_offset, 0.0+y_offset, sup_gauss_dict)
intensity = intensity_sup_pos1 + intensity_sup_pos2 + intensity_sup_pos3 + intensity_sup_pos4
plt.figure()
plt.plot(x,intensity)
plt.xlabel("Width (mm)")
plt.ylabel("Intensity ($1/I_{max}$)")
plt.savefig('LMJ_spot_type_A_quad_splitting', dpi=300, bbox_inches='tight')

intensity_sup_pos1 = nif_beam_spot_FWHM_mike(0.0+x_offset, y-y_offset, sup_gauss_dict)
intensity_sup_pos2 = nif_beam_spot_FWHM_mike(0.0-x_offset, y-y_offset, sup_gauss_dict)
intensity_sup_pos3 = nif_beam_spot_FWHM_mike(0.0+x_offset, y+y_offset, sup_gauss_dict)
intensity_sup_pos4 = nif_beam_spot_FWHM_mike(0.0-x_offset, y+y_offset, sup_gauss_dict)
intensity = intensity_sup_pos1 + intensity_sup_pos2 + intensity_sup_pos3 + intensity_sup_pos4
plt.figure()
plt.plot(x,intensity)
plt.xlabel("Width (mm)")
plt.ylabel("Intensity ($1/I_{max}$)")
plt.savefig('LMJ_spot_type_A_quad_splitting', dpi=300, bbox_inches='tight')

intensity_sup_pos1 = nif_beam_spot_FWHM_mike(X+x_offset, Y-y_offset, sup_gauss_dict)
intensity_sup_pos2 = nif_beam_spot_FWHM_mike(X-x_offset, Y-y_offset, sup_gauss_dict)
intensity_sup_pos3 = nif_beam_spot_FWHM_mike(X+x_offset, Y+y_offset, sup_gauss_dict)
intensity_sup_pos4 = nif_beam_spot_FWHM_mike(X-x_offset, Y+y_offset, sup_gauss_dict)
intensity = intensity_sup_pos1 + intensity_sup_pos2 + intensity_sup_pos3 + intensity_sup_pos4
plt.figure()
plt.pcolormesh(X, Y, intensity)
plt.xlabel("Width (mm)")
plt.ylabel("Length (mm)")
plt.gca().set_aspect('equal', adjustable='box')
plt.savefig('LMJ_spot_type_A_spot', dpi=300, bbox_inches='tight')

## LMJ-A CPP, quadsplitting, defocused 20mm

In [ ]:
# Mike Belyaev's method
icone = 0
c0 = np.array([1.0])     # Guess
c1 = np.array([0.600]) # Guess
c2 = np.array([0.400]) # Guess
c3 = np.array([2.7])

FWHMx = [0.600]
FWHMy = [0.400]
print(FWHMx[0] / np.cos(59*np.pi/180.0)) # Circular spot at 59degrees from LEH

sup_gauss_dict = super_gaussian_shape(c0[icone], c1[icone], c2[icone], c3[icone])
sup_gauss_dict["FWHMx"] = FWHMx[icone]
sup_gauss_dict["FWHMy"] = FWHMy[icone]

x_offset = 0.43
y_offset = 0.28

mesh_lim = 2.0
num_points = 1000
x = np.linspace(-mesh_lim, mesh_lim, num_points)
y = np.linspace(-mesh_lim, mesh_lim, num_points)
X, Y = np.meshgrid(x,y)

intensity_sup_pos1 = nif_beam_spot_FWHM_mike(x+x_offset, 0.0-y_offset, sup_gauss_dict)
intensity_sup_pos2 = nif_beam_spot_FWHM_mike(x-x_offset, 0.0-y_offset, sup_gauss_dict)
intensity_sup_pos3 = nif_beam_spot_FWHM_mike(x+x_offset, 0.0+y_offset, sup_gauss_dict)
intensity_sup_pos4 = nif_beam_spot_FWHM_mike(x-x_offset, 0.0+y_offset, sup_gauss_dict)
intensity = intensity_sup_pos1 + intensity_sup_pos2 + intensity_sup_pos3 + intensity_sup_pos4
plt.figure()
plt.plot(x,intensity)
plt.xlabel("Width (mm)")
plt.ylabel("Intensity ($1/I_{max}$)")
plt.savefig('LMJ_spot_type_A_quad_splitting', dpi=300, bbox_inches='tight')

intensity_sup_pos1 = nif_beam_spot_FWHM_mike(0.0+x_offset, y-y_offset, sup_gauss_dict)
intensity_sup_pos2 = nif_beam_spot_FWHM_mike(0.0-x_offset, y-y_offset, sup_gauss_dict)
intensity_sup_pos3 = nif_beam_spot_FWHM_mike(0.0+x_offset, y+y_offset, sup_gauss_dict)
intensity_sup_pos4 = nif_beam_spot_FWHM_mike(0.0-x_offset, y+y_offset, sup_gauss_dict)
intensity = intensity_sup_pos1 + intensity_sup_pos2 + intensity_sup_pos3 + intensity_sup_pos4
plt.figure()
plt.plot(x,intensity)
plt.xlabel("Width (mm)")
plt.ylabel("Intensity ($1/I_{max}$)")
plt.savefig('LMJ_spot_type_A_quad_splitting', dpi=300, bbox_inches='tight')

intensity_sup_pos1 = nif_beam_spot_FWHM_mike(X+x_offset, Y-y_offset, sup_gauss_dict)
intensity_sup_pos2 = nif_beam_spot_FWHM_mike(X-x_offset, Y-y_offset, sup_gauss_dict)
intensity_sup_pos3 = nif_beam_spot_FWHM_mike(X+x_offset, Y+y_offset, sup_gauss_dict)
intensity_sup_pos4 = nif_beam_spot_FWHM_mike(X-x_offset, Y+y_offset, sup_gauss_dict)
intensity = intensity_sup_pos1 + intensity_sup_pos2 + intensity_sup_pos3 + intensity_sup_pos4
plt.figure()
plt.pcolormesh(X, Y, intensity)
plt.xlabel("Width (mm)")
plt.ylabel("Length (mm)")
plt.gca().set_aspect('equal', adjustable='box')
plt.savefig('LMJ_spot_type_A_spot', dpi=300, bbox_inches='tight')

# Check Polar Coord Generation

In [ ]:
import healpy_pointings as hpoint

In [ ]:
nr = 10000

x = np.random.rand(nr) * 2.0 - 1.0
y = np.random.rand(nr) * 2.0 - 1.0
theta_max = np.pi / 4.0

cart_x_coords = np.zeros((nr))
cart_y_coords = np.zeros((nr))
polar_p_coords = np.zeros((nr))
polar_t_coords = np.zeros((nr))
for ir in range(nr):
    cart_x_coords[ir] = x[ir]
    cart_y_coords[ir] = y[ir]

    r, offset_phi = hpoint.square2disk(x[ir], y[ir])
    offset_theta = theta_max * r

    polar_p_coords[ir] = offset_phi
    polar_t_coords[ir] = offset_theta

X = cart_x_coords
Y = cart_y_coords

fig = plt.figure()
plt.plot(X,Y,"*")

print(np.min(polar_p_coords))
print(np.max(polar_p_coords))

U = np.degrees(polar_t_coords * np.cos(polar_p_coords))
V = np.degrees(polar_t_coords * np.sin(polar_p_coords))

fig1 = plt.figure()
plt.plot(U,V,"*")

In [ ]:
nx = 50
ny = 50

x = np.linspace(-1, 1, nx)
y = np.linspace(-1, 1, ny)
theta_max = np.pi / 4.0

cart_x_coords = np.zeros((nx,ny))
cart_y_coords = np.zeros((nx,ny))
polar_p_coords = np.zeros((nx,ny))
polar_t_coords = np.zeros((nx,ny))
for ix in range(nx):
    for iy in range(ny):
        cart_x_coords[ix,iy] = x[ix]
        cart_y_coords[ix,iy] = y[iy]
        
        r, offset_phi = hpoint.square2disk(x[ix], y[iy])
        offset_theta = theta_max * r
        
        polar_p_coords[ix,iy] = offset_phi
        polar_t_coords[ix,iy] = offset_theta

X = cart_x_coords.flatten()
Y = cart_y_coords.flatten()

fig = plt.figure()
plt.plot(X,Y,"*")

phi = polar_p_coords.flatten()
theta = polar_t_coords.flatten()
U = np.degrees(theta * np.cos(phi))
V = np.degrees(theta * np.sin(phi))

fig1 = plt.figure()
plt.plot(U,V,"*")